<a href="https://colab.research.google.com/github/Masciel-Sevilla/Segmentacion/blob/main/ComparativaFPS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Paso 1: Descomprimir el dataset (esto solo se hace una vez)
import os
import zipfile

zip_path = '/content/Balanced.zip'
extract_path = '/content/'

# Solo descomprimir si no se ha hecho antes
if not os.path.exists(os.path.join(extract_path, 'Balanced')):
    print(f"Descomprimiendo {zip_path}...")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print("¡Descompresión completada!")
else:
    print("La carpeta 'Balanced' ya existe. Omitiendo descompresión.")

Descomprimiendo /content/Balanced.zip...
¡Descompresión completada!


In [3]:
import tensorflow as tf
import numpy as np
import time
import os
from glob import glob
from tensorflow.keras import layers
import pandas as pd

class ASPPModule(layers.Layer):
    def __init__(self, filters=192, **kwargs):
        super(ASPPModule, self).__init__(**kwargs)
        self.filters = filters
        self.conv_1x1 = layers.Conv2D(filters, 1, padding='same', use_bias=False)
        self.bn_1x1 = layers.BatchNormalization()
        self.relu_1x1 = layers.ReLU()
        self.conv_3x3_6 = layers.Conv2D(filters, 3, padding='same', dilation_rate=6, use_bias=False)
        self.bn_3x3_6 = layers.BatchNormalization()
        self.relu_3x3_6 = layers.ReLU()
        self.conv_3x3_12 = layers.Conv2D(filters, 3, padding='same', dilation_rate=12, use_bias=False)
        self.bn_3x3_12 = layers.BatchNormalization()
        self.relu_3x3_12 = layers.ReLU()
        self.conv_3x3_18 = layers.Conv2D(filters, 3, padding='same', dilation_rate=18, use_bias=False)
        self.bn_3x3_18 = layers.BatchNormalization()
        self.relu_3x3_18 = layers.ReLU()
        self.global_avg_pool = layers.GlobalAveragePooling2D(keepdims=True)
        self.conv_1x1_gap = layers.Conv2D(filters, 1, padding='same', use_bias=False)
        self.bn_1x1_gap = layers.BatchNormalization()
        self.relu_1x1_gap = layers.ReLU()
        self.conv_final = layers.Conv2D(filters, 1, padding='same', use_bias=False)
        self.bn_final = layers.BatchNormalization()
        self.relu_final = layers.ReLU()
        self.dropout = layers.Dropout(0.2)

    def call(self, inputs, training=None):
        input_shape = tf.shape(inputs)
        conv_1x1 = self.relu_1x1(self.bn_1x1(self.conv_1x1(inputs), training=training))
        conv_3x3_6 = self.relu_3x3_6(self.bn_3x3_6(self.conv_3x3_6(inputs), training=training))
        conv_3x3_12 = self.relu_3x3_12(self.bn_3x3_12(self.conv_3x3_12(inputs), training=training))
        conv_3x3_18 = self.relu_3x3_18(self.bn_3x3_18(self.conv_3x3_18(inputs), training=training))
        gap = self.global_avg_pool(inputs)
        gap = self.relu_1x1_gap(self.bn_1x1_gap(self.conv_1x1_gap(gap), training=training))
        gap = tf.image.resize(gap, [input_shape[1], input_shape[2]], method='bilinear')
        concat = layers.Concatenate()([conv_1x1, conv_3x3_6, conv_3x3_12, conv_3x3_18, gap])
        output = self.relu_final(self.bn_final(self.conv_final(concat), training=training))
        output = self.dropout(output, training=training)
        return output

    def get_config(self):
        config = super(ASPPModule, self).get_config()
        config.update({"filters": self.filters})
        return config

class DeformableAttention(layers.Layer):
    def __init__(self, filters, **kwargs):
        super(DeformableAttention, self).__init__(**kwargs)
        self.filters = filters
        self.attention_conv = layers.Conv2D(self.filters, 1, padding='same', activation='sigmoid', name='attention_weights_conv', use_bias=False)
        self.bn_attention = layers.BatchNormalization()
        self.feature_conv = layers.SeparableConv2D(self.filters, 3, padding='same', name='feature_processing_conv', use_bias=False)
        self.bn_feature = layers.BatchNormalization()
        self.relu_feature = layers.ReLU()

    def call(self, inputs, training=None):
        attention_weights = self.bn_attention(self.attention_conv(inputs), training=training)
        features = self.relu_feature(self.bn_feature(self.feature_conv(inputs), training=training))
        attended_features = features * attention_weights
        return attended_features

    def get_config(self):
        config = super(DeformableAttention, self).get_config()
        config.update({"filters": self.filters})
        return config

# =============================================================================
# CONFIGURACIÓN
# =============================================================================
MODELS_TO_TEST = [
    {'name': 'Modelo B0 (EfficientNetV2-B0)', 'path': 'efficient_weed_model_B0_best.keras'},
    {'name': 'Modelo S (EfficientNetV2-S)', 'path': 'efficient_weed_model_S_best.keras'}
]
# --- ¡NUEVO! Lista de batch sizes a probar ---
BATCH_SIZES_TO_TEST = [1, 8, 16, 32, 64, 100]

TEST_IMAGES_PATH = './Balanced/test/images'
IMG_HEIGHT, IMG_WIDTH = 128, 128

# =============================================================================
# SCRIPT PRINCIPAL
# =============================================================================
print("--- Iniciando Test de Inferencia de FPS en Google Colab ---")
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"✅ GPU detectada: {gpus[0].name.split(':')[-1]}")
else:
    print("❌ No se detectó ninguna GPU.")

# Cargar todas las imágenes de prueba una sola vez
print(f"\nCargando imágenes de prueba...")
all_image_paths = sorted(glob(os.path.join(TEST_IMAGES_PATH, '*.jpg')))
all_test_images = [tf.keras.applications.efficientnet_v2.preprocess_input(tf.image.resize(tf.image.decode_jpeg(tf.io.read_file(path), channels=3), [IMG_HEIGHT, IMG_WIDTH])) for path in all_image_paths]
all_test_images = np.array(all_test_images)
print(f"Total de imágenes cargadas: {len(all_test_images)}")

# --- Almacenar resultados ---
results = []
custom_objects = {'ASPPModule': ASPPModule, 'DeformableAttention': DeformableAttention}

# Bucle para probar cada modelo
for model_info in MODELS_TO_TEST:
    model_name = model_info['name']
    model_path = model_info['path']

    print("\n" + "="*50)
    print(f"Analizando: {model_name}")
    print("="*50)

    if not os.path.exists(model_path):
        print(f"❌ ERROR: No se encontró el archivo del modelo: {model_path}. Omitiendo.")
        continue

    model = tf.keras.models.load_model(model_path, custom_objects=custom_objects, compile=False)

    # Bucle para probar cada batch size
    for batch_size in BATCH_SIZES_TO_TEST:
        print(f"\n--- Probando con Batch Size = {batch_size} ---")

        # Seleccionar el número correcto de imágenes
        num_images = (len(all_test_images) // batch_size) * batch_size
        if num_images == 0:
            print("No hay suficientes imágenes para este batch size. Omitiendo.")
            continue

        test_images_batch = all_test_images[:num_images]

        # Calentamiento
        _ = model.predict(test_images_batch[:batch_size], verbose=0, batch_size=batch_size)

        # Medición
        start_time = time.time()
        _ = model.predict(test_images_batch, verbose=0, batch_size=batch_size)
        end_time = time.time()

        total_time = end_time - start_time
        fps = num_images / total_time
        time_per_image_ms = (total_time / num_images) * 1000

        print(f"Resultados: {fps:.2f} FPS ({time_per_image_ms:.2f} ms/imagen)")
        results.append([model_name, batch_size, fps, time_per_image_ms])

# --- Mostrar tabla de resultados ---
df = pd.DataFrame(results, columns=['Modelo', 'Batch Size', 'FPS', 'Tiempo por Imagen (ms)'])
print("\n\n" + "="*60)
print("📊 RESUMEN FINAL DE RENDIMIENTO 📊")
print("="*60)
print(df.to_string(index=False))

--- Iniciando Test de Inferencia de FPS en Google Colab ---
✅ GPU detectada: 0

Cargando imágenes de prueba...
Total de imágenes cargadas: 210

Analizando: Modelo B0 (EfficientNetV2-B0)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/layer.py:421: UserWarning: `build()` was called on layer 'aspp_module_4', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/keras/src/layers/layer.py:421: UserWarning: `build()` was called on layer 'deformable_attention_4', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(



--- Probando con Batch Size = 1 ---
Resultados: 126.76 FPS (7.89 ms/imagen)

--- Probando con Batch Size = 8 ---
Resultados: 451.00 FPS (2.22 ms/imagen)

--- Probando con Batch Size = 16 ---
Resultados: 575.68 FPS (1.74 ms/imagen)

--- Probando con Batch Size = 32 ---
Resultados: 633.63 FPS (1.58 ms/imagen)

--- Probando con Batch Size = 64 ---
Resultados: 626.10 FPS (1.60 ms/imagen)

--- Probando con Batch Size = 100 ---
Resultados: 594.00 FPS (1.68 ms/imagen)

Analizando: Modelo S (EfficientNetV2-S)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/layer.py:421: UserWarning: `build()` was called on layer 'aspp_module', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/keras/src/layers/layer.py:421: UserWarning: `build()` was called on layer 'deformable_attention', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(



--- Probando con Batch Size = 1 ---
Resultados: 88.22 FPS (11.34 ms/imagen)

--- Probando con Batch Size = 8 ---
Resultados: 389.62 FPS (2.57 ms/imagen)

--- Probando con Batch Size = 16 ---
Resultados: 497.96 FPS (2.01 ms/imagen)

--- Probando con Batch Size = 32 ---
Resultados: 564.80 FPS (1.77 ms/imagen)

--- Probando con Batch Size = 64 ---
Resultados: 590.44 FPS (1.69 ms/imagen)

--- Probando con Batch Size = 100 ---
Resultados: 597.34 FPS (1.67 ms/imagen)


📊 RESUMEN FINAL DE RENDIMIENTO 📊
                       Modelo  Batch Size        FPS  Tiempo por Imagen (ms)
Modelo B0 (EfficientNetV2-B0)           1 126.757984                7.889049
Modelo B0 (EfficientNetV2-B0)           8 450.995534                2.217317
Modelo B0 (EfficientNetV2-B0)          16 575.681784                1.737071
Modelo B0 (EfficientNetV2-B0)          32 633.634504                1.578197
Modelo B0 (EfficientNetV2-B0)          64 626.099723                1.597190
Modelo B0 (EfficientNetV2-B0)       